<a href="https://colab.research.google.com/github/Aishani17/Intel-Unnati/blob/main/Teacher_Student_Kd_ngrok_.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [8]:
from google.colab import drive
drive.mount('/content/drive')

# Path to your models
#/content/drive/MyDrive/TeacherModelalz/teacher_unet_alz.h5
teacher_model_path = "/content/drive/MyDrive/TeacherModel2_intel-20250711T034015Z-1-001/TeacherModel2_intel/teacher_unet.h5"
student_model_path = "/content/drive/MyDrive/StudentModel3/stud_model_kd3.h5"


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [9]:
!pip install streamlit pyngrok --quiet


In [10]:
!ngrok config add-authtoken 2zEppHKpYo5Nge9QJVeXMxAaH84_7XcsfBjVMWqrGAMVpBefZ

Authtoken saved to configuration file: /root/.config/ngrok/ngrok.yml


In [11]:
%%writefile app.py
import streamlit as st
from tensorflow.keras.models import load_model
from PIL import Image
import numpy as np
import io

# --- Page Configuration ---
st.set_page_config(page_title="KD Demo: Teacher vs Student", layout="centered")

# --- Custom CSS Styling ---
st.markdown("""
    <style>
        .main {
            background-color: #f0f2f6;
        }
        .title {
            font-size: 40px;
            font-weight: 700;
            color: #2c3e50;
            text-align: center;
            margin-top: 30px;
        }
        .subheader {
            color: #34495e;
            font-size: 20px;
            text-align: center;
            margin-bottom: 30px;
        }
        .footer {
            font-size: 14px;
            text-align: center;
            color: #888;
            margin-top: 50px;
        }
        .uploaded-image {
            border-radius: 15px;
            box-shadow: 0px 0px 10px rgba(0,0,0,0.1);
            margin-bottom: 20px;
        }
    </style>
""", unsafe_allow_html=True)

# --- Title and Subtitle ---
st.markdown('<div class="title"> Knowledge Distillation Demo</div>', unsafe_allow_html=True)
st.markdown('<div class="subheader">Compare Teacher vs Student Model Outputs on Your Uploaded Image</div>', unsafe_allow_html=True)

# --- Load Models ---
@st.cache_resource
def load_models():
    teacher = load_model("/content/drive/MyDrive/TeacherModel2_intel-20250711T034015Z-1-001/TeacherModel2_intel/teacher_unet.h5", compile=False)
    student = load_model("/content/drive/MyDrive/StudentModel3/stud_model_kd3.h5", compile=False)
    return teacher, student

# --- Pre/Post Processing ---
def preprocess_image(image):
    resized = image.resize((128, 128))  # Resize for model input
    array = np.array(resized) / 255.0
    return np.expand_dims(array, axis=0), image.size  # return original size too

def postprocess_image(pred, original_size):
    pred = np.clip(pred[0], 0, 1) * 255
    image = Image.fromarray(pred.astype('uint8')).resize(original_size)
    return image

# --- Load Models ---
teacher_model, student_model = load_models()

# --- Upload Interface ---
uploaded_file = st.file_uploader(" Upload an image (JPG/PNG)", type=["jpg", "png"])

if uploaded_file:
    image = Image.open(uploaded_file).convert("RGB")
    st.image(image, caption="🖼️ Original Image", use_container_width=True, output_format="PNG", channels="RGB")

    with st.spinner("🔄 Processing Image..."):
        input_data, original_size = preprocess_image(image)

        col1, col2 = st.columns(2)

        with col1:
            st.subheader(" Teacher Model Output")
            teacher_output = teacher_model.predict(input_data)
            teacher_result = postprocess_image(teacher_output, original_size)
            st.image(teacher_result, use_container_width=True, caption="Teacher Output")

        with col2:
            st.subheader(" Student Model Output")
            student_output = student_model.predict(input_data)
            student_result = postprocess_image(student_output, original_size)
            st.image(student_result, use_container_width=True, caption="Student Output")

# --- Footer ---
st.markdown('<div class="footer"> Built with  using Streamlit | KD Project</div>', unsafe_allow_html=True)


Overwriting app.py


In [12]:
!streamlit run app.py &>/content/logs.txt &


In [13]:
from pyngrok import ngrok

public_url = ngrok.connect("http://localhost:8501")
print("🔗 Your Streamlit app is live at:", public_url)


🔗 Your Streamlit app is live at: NgrokTunnel: "https://cdf53124255a.ngrok-free.app" -> "http://localhost:8501"


In [ ]:
ngrok.kill()